In [2]:
from skimage import io, morphology
from skimage.color import rgb2gray
from matplotlib import pyplot as plt
import numpy as np
from skimage.transform import resize
from skimage.filters import threshold_yen

import os


In [49]:
def PreprocessImage(imgPath, index, targetDirectory):
    img = io.imread(imgPath)
    img = img [50:img.shape[0]-100, 50:img.shape[1]-50]
    #resize image
    targetSize = (1240,800)
    resizedimg = resize(img,targetSize)

    #rgp to gray
    grayimg = rgb2gray(resizedimg)

    #gray to binary
    threshold = threshold_yen(grayimg)
    binaryImg = np.where(grayimg>threshold, 1,0)
    #apply opening on image
    openinigImage = morphology.opening(binaryImg)

    #Remove large black areas
    footprint=np.ones((10,10))
    close = morphology.binary_closing(openinigImage, footprint)
    inv = np.invert(close)
    result=openinigImage+inv
    # result = morphology.binary_closing(result)
    # fp = np.array([[1,1,1],[1,1,1],[1,1,1]])
    # result = morphology.binary_opening(result, fp)
    
    #split image to 3 images
    splitImages  = [result[:600], result[240:740], result[640:]]
    
    for image in splitImages:
        #check if there is enough text in the split image
        num_zeros = (image == 0).sum()
        #check if there is text > 1% and less than 11%
        if num_zeros/(image.shape[0]*image.shape[1]) > 0.01 and num_zeros/(openinigImage.shape[0]*openinigImage.shape[1]) < 0.11:
            
            #Check if there is a large empty space
            isLargeEmptySpace = False
            for i in range(0,image.shape[0]-350,10):
                arr = image[i:i+350]
                if ((arr == 0).sum() <50):
                    isLargeEmptySpace = True 
                    break

            #save image and increment index
            if not isLargeEmptySpace:
                io.imsave(targetDirectory+str(index)+'.png', image)
                index = index+1
    
    return index

    
    


In [50]:
#Preprocess CMP Dataset images
malesDirectory = 'Dataset/Males/Males'
femalesDirectory = 'Dataset/Females/Females'
os.mkdir('results')
os.mkdir('results/Males')
os.mkdir('results/Females')

malesTargetDirectory = 'results/Males/'
index=1
for imgPath in os.listdir(malesDirectory):
    index = PreprocessImage(malesDirectory+'/'+imgPath, index, malesTargetDirectory)

femalesTargetDirectory = 'results/Females/'
index=1
for imgPath in os.listdir(femalesDirectory):
    index = PreprocessImage(femalesDirectory+'/'+imgPath, index, femalesTargetDirectory)
    




C:\Users\ammar\AppData\Local\Temp/ipykernel_19660/3057642245.py:45: UserWarning: results/Males/1.png is a low contrast image
  io.imsave(targetDirectory+str(index)+'.png', image)
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\ammar\AppData\Local\Temp/ipykernel_19660/3057642245.py:45: UserWarning: results/Males/2.png is a low contrast image
  io.imsave(targetDirectory+str(index)+'.png', image)
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\ammar\AppData\Local\Temp/ipykernel_19660/3057642245.py:45: UserWarning: results/Males/3.png is a low contrast image
  io.imsave(targetDirectory+str(index)+'.png', image)
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\ammar\AppData\Local\Temp/ipykernel_19660/3057642245.py:45: UserWarning: results/Males/4.png is a low co

C:\Users\ammar\AppData\Local\Temp/ipykernel_19660/3057642245.py:45: UserWarning: results/Females/1.png is a low contrast image
  io.imsave(targetDirectory+str(index)+'.png', image)
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\ammar\AppData\Local\Temp/ipykernel_19660/3057642245.py:45: UserWarning: results/Females/2.png is a low contrast image
  io.imsave(targetDirectory+str(index)+'.png', image)
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\ammar\AppData\Local\Temp/ipykernel_19660/3057642245.py:45: UserWarning: results/Females/3.png is a low contrast image
  io.imsave(targetDirectory+str(index)+'.png', image)
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\ammar\AppData\Local\Temp/ipykernel_19660/3057642245.py:45: UserWarning: results/Females/4.png is 